# Quantizing RNN Models

In [1]:
import sys
import pymysql
sys.path.append("../..")
page_G="Collect_activation_statistics"
def sqllog(strin):
    global page_G
    strin_=str(strin)
    strin_=strin_.replace("'", '"')
    #資料庫連線設定
    db = pymysql.connect(host='localhost', port=3306, user='admin_user', passwd='admin', db='ai23', charset='utf8')
    #建立操作游標
    cursor = db.cursor()
    #SQL語法
    sql = "INSERT INTO `ai23`.`"+str(page_G)+"` (`str`) VALUES ('"+str(strin_)+"');"
    #執行語法

    try:
      cursor.execute(sql)
      #提交修改
      db.commit()
      #print('success')
    except:
      #發生錯誤時停止執行SQL
      db.rollback()
      print('SQLerror')

    #關閉連線
    db.close()

In this example, we show how to quantize recurrent models.  
Using a pretrained model `model.RNNModel`, we convert the built-in pytorch implementation of LSTM to our own, modular implementation.  
The pretrained model was generated with:  
```time python3 main.py --cuda --emsize 1500 --nhid 1500 --dropout 0.65 --tied --wd=1e-6```  
The reason we replace the LSTM that is because the inner operations in the pytorch implementation are not accessible to us, but we still want to quantize these operations. <br />
Afterwards we can try different techniques to quantize the whole model.  

In [2]:
from model import DistillerRNNModel, RNNModel
from data import Corpus
import torch
from torch import nn
import distiller
from distiller.modules import DistillerLSTM as LSTM
from tqdm import tqdm # for pretty progress bar
import numpy as np
from copy import deepcopy

import warnings
warnings.filterwarnings(action='default', module='distiller.quantization')
warnings.filterwarnings(action='default', module='distiller.quantization.range_linear')

/home/macroai1/anaconda3/envs/ai23_37/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/home/macroai1/anaconda3/envs/ai23_37/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.

### Preprocess the data

In [3]:
corpus = Corpus('./data/wikitext-2/')

In [4]:
def batchify(data, bsz):
    # Work out how cleanly we can divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)
device = 'cuda:0'
batch_size = 20
eval_batch_size = 10
train_data = batchify(corpus.train, batch_size)
val_data = batchify(corpus.valid, eval_batch_size)
test_data = batchify(corpus.test, eval_batch_size)

### Loading the model and converting to our own implementation

In [5]:
rnn_model = torch.load('./checkpoint.pth.tar.best')
rnn_model = rnn_model.to(device)
rnn_model

RNNModel(
  (drop): Dropout(p=0.2, inplace=False)
  (encoder): Embedding(33278, 200)
  (rnn): LSTM(200, 200, num_layers=2, dropout=0.2)
  (decoder): Linear(in_features=200, out_features=33278, bias=True)
)

Here we convert the pytorch LSTM implementation to our own, by calling `LSTM.from_pytorch_impl`:

In [6]:
def manual_model(pytorch_model_: RNNModel):
    nlayers, ninp, nhid, ntoken, tie_weights = \
        pytorch_model_.nlayers, \
        pytorch_model_.ninp, \
        pytorch_model_.nhid, \
        pytorch_model_.ntoken, \
        pytorch_model_.tie_weights

    model = DistillerRNNModel(nlayers=nlayers, ninp=ninp, nhid=nhid, ntoken=ntoken, tie_weights=tie_weights).to(device)
    model.eval()
    model.encoder.weight = nn.Parameter(pytorch_model_.encoder.weight.clone().detach())
    model.decoder.weight = nn.Parameter(pytorch_model_.decoder.weight.clone().detach())
    model.decoder.bias = nn.Parameter(pytorch_model_.decoder.bias.clone().detach())
    model.rnn = LSTM.from_pytorch_impl(pytorch_model_.rnn)

    return model

man_model = manual_model(rnn_model)
torch.save(man_model, 'manual.checkpoint.pth.tar')
man_model

DistillerRNNModel(
  (encoder): Embedding(33278, 200)
  (rnn): DistillerLSTM(200, 200, num_layers=2, dropout=0.20, bidirectional=False)
  (decoder): Linear(in_features=200, out_features=33278, bias=True)
)

### Batching the data for evaluation

In [7]:
sequence_len = 35
def get_batch(source, i):
    seq_len = min(sequence_len, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target

hidden = rnn_model.init_hidden(eval_batch_size)
data, targets = get_batch(test_data, 0)

### Check that the convertion has succeeded

In [8]:
rnn_model.eval()
man_model.eval()
y_t, h_t = rnn_model(data, hidden)
y_p, h_p = man_model(data, hidden)

print("Max error in y: %f" % (y_t-y_p).abs().max().item())

!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
Max error in y: 0.000005


### Defining the evaluation

In [9]:
criterion = nn.CrossEntropyLoss()
def repackage_hidden(h):
    """Wraps hidden states in new Tensors, to detach them from their history."""
    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)
    

def evaluate(model, data_source):
    # Turn on evaluation mode which disables dropout.
    model.eval()
    total_loss = 0.
    ntokens = len(corpus.dictionary)
    hidden = model.init_hidden(eval_batch_size)
    with torch.no_grad():
        with tqdm(range(0, data_source.size(0), sequence_len)) as t:
            # The line below was fixed as per: https://github.com/pytorch/examples/issues/214
            for i in t:
                data, targets = get_batch(data_source, i)
                output, hidden = model(data, hidden)
                #sqllog(output)
                output_flat = output.view(-1, ntokens)
                total_loss += len(data) * criterion(output_flat, targets).item()
                hidden = repackage_hidden(hidden)
                avg_loss = total_loss / (i + 1)
                t.set_postfix((('val_loss', avg_loss), ('ppl', np.exp(avg_loss))))
    return total_loss / len(data_source)

# Quantizing the Model

## Collect activation statistics

The model uses activation statistics to determine how big the quantization range is. The bigger the range - the larger the round off error after quantization which leads to accuracy drop.  
Our goal is to minimize the range s.t. it contains the absolute most of our data.  
After that, we divide the range into chunks of equal size, according to the number of bits, and transform the data according to this scale factor.  
Read more on scale factor calculation [in our docs](https://intellabs.github.io/distiller/algo_quantization.html).

The class `QuantCalibrationStatsCollector` collects the statistics for defining the range $r = max - min$.  

Each forward pass, the collector records the values of inputs and outputs, for each layer:
- absolute over all batches min, max (stored in `min`, `max`)
- average over batches, per batch min, max (stored in `avg_min`, `avg_max`)
- mean
- std
- shape of output tensor  

All these values can be used to define the range of quantization, e.g. we can use the absolute `min`, `max` to define the range.

In [10]:
import os
from distiller.data_loggers import collect_quant_stats, QuantCalibrationStatsCollector

man_model = torch.load('./manual.checkpoint.pth.tar')
distiller.utils.assign_layer_fq_names(man_model)
collector = QuantCalibrationStatsCollector(man_model)

stats_file = './acts_quantization_stats.yaml'

if not os.path.isfile(stats_file):
    def eval_for_stats(model):
        evaluate(man_model, val_data)
    collect_quant_stats(man_model, eval_for_stats, save_dir='.')

## Prepare the Model For Quantization
  
We quantize the model after the training has completed.  
Here we check the baseline model perplexity, to have an idea how good the quantization is.

In [11]:
#page_G="Prepare_the_Model_For_Quantization"

In [12]:
from distiller.quantization import PostTrainLinearQuantizer, LinearQuantMode
from copy import deepcopy

# Load and evaluate the baseline model.
man_model = torch.load('./manual.checkpoint.pth.tar')
val_loss = evaluate(man_model, val_data)
print('val_loss:%8.2f\t|\t ppl:%8.2f' % (val_loss, np.exp(val_loss)))

  0%|▏                                      | 2/622 [00:00<00:37, 16.45it/s, val_loss=9.49, ppl=1.32e+4]

!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!


  1%|▎                                          | 4/622 [00:00<01:02,  9.87it/s, val_loss=6.11, ppl=449]

!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!


  1%|▌                                          | 8/622 [00:00<00:52, 11.74it/s, val_loss=5.51, ppl=246]

...

!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!


100%|████████████████████████████████████████▊| 619/622 [00:32<00:00, 20.94it/s, val_loss=4.95, ppl=141]

!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!


100%|█████████████████████████████████████████| 622/622 [00:32<00:00, 22.16it/s, val_loss=4.95, ppl=141]

!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
val_loss:    4.94	|	 ppl:  139.89


Now we do our magic - __Preparing the model for quantization__.  
The quantizer replaces the layers in out model with their quantized versions.  

In [13]:
# Define the quantizer
quantizer = PostTrainLinearQuantizer(
    deepcopy(man_model),
    model_activation_stats=stats_file)

# Quantizer magic
stats_before_prepare = deepcopy(quantizer.model_activation_stats)
dummy_input = (torch.zeros(1,1).to(dtype=torch.long), man_model.init_hidden(1))
quantizer.prepare_model(dummy_input)

!debug!
!debug!
!debug!
!debug!


### Net-Aware Quantization

Note that we passes a dummy input to `prepare_model`. This is required for the quantizer to be able to create a graph representation of the model, and to infer the connectivity between the modules.  
Understanding the connectivity of the model is required to enable **"Net-aware quantization"**. This term (coined in [\[1\]](#references), section 3.2.2), means we can achieve better quantization by considering sequences of operations.  
In the case of LSTM, we have an element-wise add operation whose output is split into 4 and fed into either Tanh or Sigmoid activations. Both of these ops saturate at relatively small input values - tanh at approximately $|4|$, and sigmoid saturates at approximately $|6|$. This means we can safely clip the output of the element-wise add operation between $[-6,6]$. `PostTrainLinearQuantizer` detects this patterm and modifies the statistics accordingly.

In [14]:
import pprint
pp = pprint.PrettyPrinter(indent=1)
print('Stats BEFORE prepare_model:')
pp.pprint(stats_before_prepare['rnn.cells.0.eltwiseadd_gate']['output'])

print('\nStats AFTER to prepare_model:')
pp.pprint(quantizer.model_activation_stats['rnn.cells.0.eltwiseadd_gate']['output'])

Stats BEFORE prepare_model:
OrderedDict([('min', -15.612003326416016),
             ('max', 15.450967788696289),
             ('avg_min', -6.393781979404043),
             ('avg_max', 5.522592915806375),
             ('mean', -0.7798242293363614),
             ('std', 1.3385719958875721),
             ('shape', '(10, 6000)')])

Stats AFTER to prepare_model:
OrderedDict([('min', -6.0),
             ('max', 6.0),
             ('avg_min', -6.0),
             ('avg_max', 5.522592915806375),
             ('mean', -0.7798242293363614),
             ('std', 1.3385719958875721),
             ('shape', '(10, 6000)')])


Note the value for `avg_max` did not change, since it was already below the clipping value of $6.0$.

### Inspecting the Quantized Model

Let's see how the model has after being prepared for quantization:

In [15]:
quantizer.model

DistillerRNNModel(
  (encoder): RangeLinearEmbeddingWrapper(
    (wrapped_module): Embedding(33278, 200)
  )
  (rnn): DistillerLSTM(200, 200, num_layers=2, dropout=0.20, bidirectional=False)
  (decoder): RangeLinearQuantParamLayerWrapper(
    weights_quant_settings=(num_bits=8 ; quant_mode=SYMMETRIC ; clip_mode=NONE ; clip_n_stds=None ; clip_half_range=False ; per_channel=False)
    output_quant_settings=(num_bits=8 ; quant_mode=SYMMETRIC ; clip_mode=NONE ; clip_n_stds=None ; clip_half_range=False ; per_channel=False)
    accum_quant_settings=(num_bits=32 ; quant_mode=SYMMETRIC ; clip_mode=NONE ; clip_n_stds=None ; clip_half_range=False ; per_channel=False)
      inputs_quant_auto_fallback=True, forced_quant_settings_for_inputs=None
    scale_approx_mult_bits=None
    preset_activation_stats=True
      output_scale=3.670504, output_zero_point=0.000000
    weights_scale=38.034809, weights_zero_point=0.000000
    (wrapped_module): Linear(in_features=200, out_features=33278, bias=True)
  

Note how `encoder` and `decoder` have been replaced with wrapper layers (for the relevant module type), which handle the quantization. The same holds for the internal layers of the `DistillerLSTM` module, which we don't print for brevity sake. To "peek" inside the `DistillerLSTM` module, we need to access it directly. As an example, let's take a look at a couple of the internal layers:

In [16]:
print(quantizer.model.rnn.cells[0].fc_gate_x)
print(quantizer.model.rnn.cells[0].eltwiseadd_gate)

RangeLinearQuantParamLayerWrapper(
  weights_quant_settings=(num_bits=8 ; quant_mode=SYMMETRIC ; clip_mode=NONE ; clip_n_stds=None ; clip_half_range=False ; per_channel=False)
  output_quant_settings=(num_bits=8 ; quant_mode=SYMMETRIC ; clip_mode=NONE ; clip_n_stds=None ; clip_half_range=False ; per_channel=False)
  accum_quant_settings=(num_bits=32 ; quant_mode=SYMMETRIC ; clip_mode=NONE ; clip_n_stds=None ; clip_half_range=False ; per_channel=False)
    inputs_quant_auto_fallback=True, forced_quant_settings_for_inputs=None
  scale_approx_mult_bits=None
  preset_activation_stats=True
    output_scale=17.780996, output_zero_point=0.000000
  weights_scale=29.729475, weights_zero_point=0.000000
  (wrapped_module): Linear(in_features=200, out_features=800, bias=True)
)
RangeLinearQuantEltwiseAddWrapper(
  output_quant_settings=(num_bits=8 ; quant_mode=SYMMETRIC ; clip_mode=NONE ; clip_n_stds=None ; clip_half_range=False ; per_channel=False)
  accum_quant_settings=(num_bits=32 ; quant_mode

## Running the Quantized Model

### Try 1: Initial settings - simple symmetric quantization

Finally, let's go ahead and evaluate the quantized model:

In [17]:
#page_G="Try_1"

In [18]:
val_loss = evaluate(quantizer.model.to(device), val_data)
print('val_loss:%8.2f\t|\t ppl:%8.2f' % (val_loss, np.exp(val_loss)))

  0%|                                                                           | 0/622 [00:00<?, ?it/s]

!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!


  0%|                                       | 1/622 [00:00<03:31,  2.93it/s, val_loss=168, ppl=7.94e+72]

!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!


  0%|▏                                       | 2/622 [00:00<03:21,  3.07it/s, val_loss=9.68, ppl=1.6e+4]

...

!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!


100%|████████████████████████████████████████▉| 621/622 [02:49<00:00,  3.86it/s, val_loss=5.09, ppl=163]

!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!


100%|█████████████████████████████████████████| 622/622 [02:49<00:00,  4.14it/s, val_loss=5.09, ppl=162]

!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
val_loss:    5.08	|	 ppl:  161.34


As we can see here, the perplexity has increased much - meaning our quantization has damaged the accuracy of our model.

### Try 2: Assymetric, per-channel

Let's try quantizing each channel separately, and making the range of the quantization asymmetric.

In [19]:
quantizer = PostTrainLinearQuantizer(
    deepcopy(man_model),
    model_activation_stats=stats_file,
    mode=LinearQuantMode.ASYMMETRIC_SIGNED,
    per_channel_wts=True
)
quantizer.prepare_model(dummy_input)
quantizer.model

!debug!
!debug!
!debug!
!debug!


DistillerRNNModel(
  (encoder): RangeLinearEmbeddingWrapper(
    (wrapped_module): Embedding(33278, 200)
  )
  (rnn): DistillerLSTM(200, 200, num_layers=2, dropout=0.20, bidirectional=False)
  (decoder): RangeLinearQuantParamLayerWrapper(
    weights_quant_settings=(num_bits=8 ; quant_mode=ASYMMETRIC_SIGNED ; clip_mode=NONE ; clip_n_stds=None ; clip_half_range=False ; per_channel=True)
    output_quant_settings=(num_bits=8 ; quant_mode=ASYMMETRIC_SIGNED ; clip_mode=NONE ; clip_n_stds=None ; clip_half_range=False ; per_channel=False)
    accum_quant_settings=(num_bits=32 ; quant_mode=SYMMETRIC ; clip_mode=NONE ; clip_n_stds=None ; clip_half_range=False ; per_channel=False)
      inputs_quant_auto_fallback=True, forced_quant_settings_for_inputs=None
    scale_approx_mult_bits=None
    preset_activation_stats=True
      output_scale=5.024112, output_zero_point=48.000000
    weights_scale=PerCh, weights_zero_point=PerCh
    (wrapped_module): Linear(in_features=200, out_features=33278, bias

In [20]:
#page_G="Try_2"

In [21]:
val_loss = evaluate(quantizer.model.to(device), val_data)
print('val_loss:%8.2f\t|\t ppl:%8.2f' % (val_loss, np.exp(val_loss)))

  0%|                                                                           | 0/622 [00:00<?, ?it/s]

!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!


  0%|                                       | 1/622 [00:00<03:02,  3.41it/s, val_loss=167, ppl=4.45e+72]

!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!


  0%|▏                                      | 2/622 [00:00<02:56,  3.50it/s, val_loss=9.65, ppl=1.55e+4]

...

  7%|███▏                                       | 46/622 [00:13<02:33,  3.76it/s, val_loss=5.1, ppl=164]

!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!


  8%|███▏                                      | 47/622 [00:13<02:30,  3.83it/s, val_loss=5.11, ppl=166]

!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!
!debug!


KeyboardInterrupt: 

A tiny bit better, but still no good.

### Try 3: Mixed FP16 and INT8

Let us try the half precision (aka FP16) version of the model:

In [ ]:
#page_G="Try_3"

In [ ]:
model_fp16 = deepcopy(man_model).half()
val_loss = evaluate(model_fp16, val_data)
print('val_loss: %8.6f\t|\t ppl:%8.2f' % (val_loss, np.exp(val_loss)))

The result is very close to our original model! That means that the roundoff when quantizing linearly to 8-bit integers is what hurts our accuracy.

Luckily, `PostTrainLinearQuantizer` supports quantizing some/all layers to FP16 using the `fp16` parameter. In light of what we just saw, and as stated in [\[2\]](#References), let's try keeping element-wise operations at FP16, and quantize everything else to 8-bit using the same settings as in try 2.

In [ ]:
overrides_yaml = """
.*eltwise.*:
    fp16: true
encoder:
    fp16: true
decoder:
    fp16: true
"""
overrides = distiller.utils.yaml_ordered_load(overrides_yaml)
quantizer = PostTrainLinearQuantizer(
    deepcopy(man_model),
    model_activation_stats=stats_file,
    mode=LinearQuantMode.ASYMMETRIC_SIGNED,
    overrides=overrides,
    per_channel_wts=True
)
quantizer.prepare_model(dummy_input)
quantizer.model

In [ ]:
#page_G="Try_32"

In [ ]:
val_loss = evaluate(quantizer.model.to(device), val_data)
print('val_loss:%8.6f\t|\t ppl:%8.2f' % (val_loss, np.exp(val_loss)))

The accuracy is still holding up very well, even though we quantized the inner linear layers!  

### Try 4: Clipping Activations

Now, lets try to choose different boundaries for `min`, `max`.  
Instead of using absolute ones, we take the average of all batches (`avg_min`, `avg_max`), which is an indication of where usually most of the boundaries lie. This is done by specifying the `clip_acts` parameter to `ClipMode.AVG` or `"AVG"` in the quantizer ctor:

In [ ]:
#page_G="Try_4"

In [ ]:
overrides_yaml = """
encoder:
    fp16: true
decoder:
    fp16: true
"""
overrides = distiller.utils.yaml_ordered_load(overrides_yaml)
quantizer = PostTrainLinearQuantizer(
    deepcopy(man_model),
    model_activation_stats=stats_file,
    mode=LinearQuantMode.ASYMMETRIC_SIGNED,
    overrides=overrides,
    per_channel_wts=True,
    clip_acts="AVG"
)
quantizer.prepare_model(dummy_input)
val_loss = evaluate(quantizer.model.to(device), val_data)
print('val_loss:%8.6f\t|\t ppl:%8.2f' % (val_loss, np.exp(val_loss)))

Great! Even though we quantized all of the layers except the embedding and the decoder - we got almost no accuracy penalty. Lets try quantizing them as well:

In [ ]:
#page_G="Try_f"

In [ ]:
quantizer = PostTrainLinearQuantizer(
    deepcopy(man_model),
    model_activation_stats=stats_file,
    mode=LinearQuantMode.ASYMMETRIC_SIGNED,
    per_channel_wts=True,
    clip_acts="AVG"
)
quantizer.prepare_model(dummy_input)
val_loss = evaluate(quantizer.model.to(device), val_data)
print('val_loss:%8.6f\t|\t ppl:%8.2f' % (val_loss, np.exp(val_loss)))

In [ ]:
quantizer.model

Here we see that sometimes quantizing with the right boundaries gives better results than actually using floating point operations (even though they are half precision). 

## Conclusion

Choosing the right boundaries for quantization  was crucial for achieving almost no degradation in accrucay of LSTM.  
  
Here we showed how to use the Distiller quantization API to quantize an RNN model, by converting the PyTorch implementation into a modular one and then quantizing each layer separately.

## References

1. **Jongsoo Park, Maxim Naumov, Protonu Basu, Summer Deng, Aravind Kalaiah, Daya Khudia, James Law, Parth Malani, Andrey Malevich, Satish Nadathur, Juan Miguel Pino, Martin Schatz, Alexander Sidorov, Viswanath Sivakumar, Andrew Tulloch, Xiaodong Wang, Yiming Wu, Hector Yuen, Utku Diril, Dmytro Dzhulgakov, Kim Hazelwood, Bill Jia, Yangqing Jia, Lin Qiao, Vijay Rao, Nadav Rotem, Sungjoo Yoo, Mikhail Smelyanskiy**. Deep Learning Inference in Facebook Data Centers: Characterization, Performance Optimizations and Hardware Implications. [arxiv:1811.09886](https://arxiv.org/abs/1811.09886)

2. **Qinyao He, He Wen, Shuchang Zhou, Yuxin Wu, Cong Yao, Xinyu Zhou, Yuheng Zou**. Effective Quantization Methods for Recurrent Neural Networks. [arxiv:1611.10176](https://arxiv.org/abs/1611.10176)